# Si/Si3N4 - Approximate

In [3]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index, get_disorientation
from numpy import array, dot, round, cross, eye
from numpy.linalg import inv, det, norm

In [4]:
#https://aip.scitation.org/doi/full/10.1063/1.3489687

In [5]:
my_interface = core('cif_files/Si_mp-149_conventional_standard.cif',\
                    'cif_files/Si3N4_mp-988_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


/Users/jasonxie/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [6]:
R = get_disorientation(L1 = my_interface.conv_lattice_1, L2 = my_interface.conv_lattice_2, \
                       v1 = [-1,1,0], hkl1 = [1, 1, 1], \
                       v2 = [1,0,0], hkl2 = [0, 0, 1])
R

array([[-7.07106781e-01, -4.08248290e-01,  5.77350269e-01],
       [ 7.07106781e-01, -4.08248290e-01,  5.77350269e-01],
       [-3.53525080e-17,  8.16496581e-01,  5.77350269e-01]])

## Get the two axis aligned to be collinear, expressed in primitive cell

Here, we want the (1,-1,0,2) plane of the Al2O3 coincident whith (1,1,-2,0) plane of ZnO.

Firstly, the programme rotate lattice 2 so that the two lattice plane overlap.

As the normal of the plane have different indices with the miller indices, we need to compute it.

In [7]:
my_interface.lattice_2

array([[ 0.00000000e+00, -3.83015393e+00, -3.83015393e+00],
       [ 0.00000000e+00, -6.63402121e+00,  6.63402121e+00],
       [-2.92508500e+00, -9.38117150e-16,  4.69058575e-16]])

In [8]:
hkl_1 = get_primitive_hkl([1, 1, 1], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_2, my_interface.lattice_2)

## Searching two_dimensional CSL

In [9]:
my_interface.parse_limit(du = 1e-1, S  = 1e-1, sgm1=100, sgm2=100, dd = 1e-1)

#Do searching!
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 2, \
                                    dtheta = 0.01, pre_R = R, match_tol = 0.01, pre_dt= True)

Congrates, we found an appx CSL!

U1 = 
[[ 0 -2]
 [-2  2]
 [ 2  0]]; sigma_1 = 7

U2 = 
[[ 0  0]
 [ 1  0]
 [ 1 -1]]; sigma_2 = 1

D = 
[[ 1.00640892 -0.00320446 -0.00320446]
 [-0.00320446  1.00640892 -0.00320446]
 [-0.00320446 -0.00320446  1.00640892]]

axis = [-7.47674648 -7.47674648 -7.47674648] ; theta = 0.0



## Get bicrystal

In [10]:
my_interface.compute_bicrystal_two_D(hkl_1 = hkl_1, hkl_2=hkl_2, normal_ortho = True, plane_ortho = False, lim = 50, tol_ortho = 1e-2, tol_integer=1e-3)

cell 1:
[[-1  0 -2]
 [-1 -2  2]
 [-1  2  0]]
cell 2:
[[-1  0  0]
 [ 0  1  0]
 [ 0  1 -1]]


In [28]:
my_interface.get_bicrystal(two_D = True, xyz_1 = [1,1,1], xyz_2 = [1,1,1])

Please check the POSCAR file